In [26]:
def SystemMessageCallback(serial, prnt_msg: bool = True, ret_hex_int: int = 0):
    """Reads the message buffer of a serial connection. Also prints out the general system message.
    serial      ... serial connection
    prnt_msg    ... print out the buffer
    ret_hex_int ... Parameters -> ['none','hex', 'int', 'both']

    """
    msg_dict = {
        "0x01": "No message inside the message buffer",
        "0x02": "Timeout: Communication-timeout (less data than expected)",
        "0x04": "Wake-Up Message: System boot ready",
        "0x11": "TCP-Socket: Valid TCP client-socket connection",
        "0x81": "Not-Acknowledge: Command has not been executed",
        "0x82": "Not-Acknowledge: Command could not be recognized",
        "0x83": "Command-Acknowledge: Command has been executed successfully",
        "0x84": "System-Ready Message: System is operational and ready to receive data",
        "0x91": "Data holdup: Measurement data could not be sent via the master interface",
    }
    timeout_count = 0
    received = []
    received_hex = []
    data_count = 0

    while True:
        buffer = serial.read()
        if buffer:
            received.extend(buffer)
            data_count += len(buffer)
            timeout_count = 0
            continue
        timeout_count += 1
        if timeout_count >= 1:
            # Break if we haven't received any data
            break

        received = "".join(str(received))  # If you need all the data
    received_hex = [hex(receive) for receive in received]
    try:
        msg_idx = received_hex.index("0x18")
        print(msg_dict[received_hex[msg_idx + 2]])
    except BaseException:
        print(msg_dict["0x01"])
        prnt_msg = False
    if prnt_msg:
        print("message buffer:\n", received_hex)
        print("message length:\t", data_count)

    if ret_hex_int == "none":
        return
    elif ret_hex_int == "hex":
        return received_hex
    elif ret_hex_int == "int":
        return received
    elif ret_hex_int == "both":
        return received, received_hex

In [2]:
import time

import numpy as np

import sciopy as sc
from sciopy.setup_m import SystemMessageCallback

In [3]:
sc.available_serial_ports()

['COM1', 'COM3']

In [4]:
ser = sc.connect_COM_port("COM3", timeout=1)

Connection to COM3 is established.


In [5]:
def GetFirmwareIDs(serial):
    """Get firmware IDs"""
    serial.write(bytearray([0xD2, 0x00, 0xD2]))
    SystemMessageCallback(serial)

In [6]:
GetFirmwareIDs(ser)

message buffer:
 ['0xd2', '0x1e', '0x1', '0x11', '0x0', '0x19', '0x0', '0xb', '0x40', '0x0', '0x9', '0x0', '0x25', '0x2', '0xc9', '0x0', '0xf', '0x1', '0x11', '0x0', '0x19', '0x0', '0xb', '0x40', '0x0', '0x9', '0x0', '0x25', '0x2', '0xc9', '0x0', '0xf', '0xd2', '0x18', '0x1', '0x83', '0x18']
message length:	 33
Command-Acknowledge: Command has been executed successfully


In [9]:
def SoftwareReset(serial):
    serial.write(bytearray([0xA1, 0x00, 0xA1]))
    SystemMessageCallback(serial)

In [10]:
SoftwareReset(ser)

message buffer:
 ['0x18', '0x1', '0x83', '0x18']
message length:	 0
Command-Acknowledge: Command has been executed successfully


In [26]:
def ResetMeasurementSetup(serial):
    serial.write(bytearray([0xB0, 0x01, 0x01, 0xB0]))
    SystemMessageCallback(serial)

In [27]:
ResetMeasurementSetup(ser)

SerialException: WriteFile failed (PermissionError(13, 'Das Gerät erkennt den Befehl nicht.', None, 22))

# Final Running schedule

In [36]:
def Config_01(serial) -> None:
    serial.write(bytearray([0xB0, 0x01, 0x01, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x02, 0x00, 0x01, 0xB0]))  # Burst count
    # SystemMessageCallback(ser)
    serial.write(
        bytearray(
            [0xB0, 0x09, 0x05, 0x3F, 0x50, 0x62, 0x4D, 0xD2, 0xF1, 0xA9, 0xFC, 0xB0]
        )
    )
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x02, 0x0D, 0x01, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x09, 0x01, 0x00, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x08, 0x01, 0x01, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x02, 0x0C, 0x01, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x05, 0x03, 0x40, 0x00, 0x00, 0x00, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(
        bytearray(
            [
                0xB0,
                0x0C,
                0x04,
                0x44,
                0x7A,
                0x00,
                0x00,
                0x44,
                0x7A,
                0x00,
                0x00,
                0x00,
                0x01,
                0x00,
                0xB0,
            ]
        )
    )
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x01, 0x02, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x02, 0x03, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x03, 0x04, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x04, 0x05, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x05, 0x06, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x06, 0x07, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x07, 0x08, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x08, 0x09, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x09, 0x0A, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x0A, 0x0B, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x0B, 0x0C, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x0C, 0x0D, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x0D, 0x0E, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x0E, 0x0F, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x0F, 0x10, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x10, 0x11, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x11, 0x12, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x12, 0x13, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x13, 0x14, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x14, 0x15, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x15, 0x16, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x16, 0x17, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x17, 0x18, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x18, 0x19, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x19, 0x1A, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x1A, 0x1B, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x1B, 0x1C, 0xB0]))
    # SystemMessageCallback
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x1C, 0x1D, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x1D, 0x1E, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x1E, 0x1F, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x1F, 0x20, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x20, 0x01, 0xB0]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB1, 0x01, 0x03, 0xB1]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB2, 0x02, 0x01, 0x01, 0xB2]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB2, 0x02, 0x03, 0x01, 0xB2]))
    # SystemMessageCallback(ser)
    serial.write(bytearray([0xB2, 0x02, 0x02, 0x01, 0xB2]))
    # SystemMessageCallback(ser)
    SystemMessageCallback(ser)
    # serial.write(bytearray([0xB4, 0x01, 0x01, 0xB4]))
    # sleep(10)
    # serial.write(bytearray([0xB4, 0x01, 0x00, 0xB4]))

In [46]:
def StartStopMeasurement(serial):
    serial.write(bytearray([0xB4, 0x01, 0x01, 0xB4]))
    measurement_data_int, measurement_data_hex = SystemMessageCallback(
        serial, ret_hex_int="both"
    )
    print("Stopping measurement:")
    serial.write(bytearray([0xB4, 0x01, 0x00, 0xB4]))
    SystemMessageCallback(serial, ret_hex_int="int")
    return measurement_data_int, measurement_data_hex


def StopMeasurement(serial):
    serial.write(bytearray([0xB4, 0x01, 0x00, 0xB4]))
    SystemMessageCallback(serial)

In [47]:
Config_01(ser)

Command-Acknowledge: Command has been executed successfully
message buffer:
 ['0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', '0x18', '0x18', '0x1', '0x83', 

In [48]:
SystemMessageCallback(ser)

No message inside the message buffer


In [ ]:
measurement_data_int, measurement_data_hex = StartStopMeasurement(ser)

In [52]:
np.savez(
    "test_messung_burstCnt_1.npz",
    data_hex=measurement_data_hex,
    data_int=measurement_data_int,
)

In [18]:
StopMeasurement(ser)

start tag: b'\x02'
msg_new len: 3
None
None


In [41]:
ser.read()

b'\xc0'

In [ ]:
mx = 0
while ser.read() != None:  # 4000+
    ser.read()
    mx += 1
print(mx)

In [6]:
int.from_bytes(b"0x01", byteorder="little")

825260080

In [11]:
int.from_bytes(bytes(0xA2), byteorder="little")

0

In [13]:
int(0x18)

24